# Apple (AAPL) Stock Time-Series Analysis (Jan 2022 - Jan 2023)

**Course:** Intro to Data & Programming (BUAN-651)  
**Built:** Dec 2023  

This notebook downloads AAPL historical price data, computes descriptive statistics across multiple time granularities (daily/weekly/monthly/quarterly), visualizes trends (line + weekly candlestick), and flags simple buy/sell signals using a 20-day moving average.

> Note: `yfinance` requires an internet connection to fetch Yahoo Finance data.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

import plotly.graph_objects as go


In [ ]:
# Parameters
TICKER = 'AAPL'
START_DATE = '2022-01-01'
END_DATE = '2023-01-01'

def download_stock_data(ticker: str, start_date: str, end_date: str) -> pd.DataFrame:
    """Download OHLCV data from Yahoo Finance via yfinance."""
    df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
    df.index = pd.to_datetime(df.index)
    return df

stock_data = download_stock_data(TICKER, START_DATE, END_DATE)
stock_data.head()


In [ ]:
# Descriptive statistics (daily close)
daily_stats = stock_data['Close'].describe()
daily_stats

# Resampled statistics
weekly_stats = stock_data['Close'].resample('W-FRI').agg(['mean','std','min','max'])
monthly_stats = stock_data['Close'].resample('M').agg(['mean','std','min','max'])
quarterly_stats = stock_data['Close'].resample('Q').agg(['mean','std','min','max'])

display(weekly_stats.head(), monthly_stats.head(), quarterly_stats)


In [ ]:
# Line chart: daily close
plt.figure(figsize=(10, 5))
plt.plot(stock_data['Close'], label='Daily Close')
plt.title('AAPL Daily Closing Prices')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Weekly candlestick (true weekly OHLC)
weekly_ohlc = stock_data.resample('W-FRI').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).dropna()

fig = go.Figure(data=[go.Candlestick(
    x=weekly_ohlc.index,
    open=weekly_ohlc['Open'],
    high=weekly_ohlc['High'],
    low=weekly_ohlc['Low'],
    close=weekly_ohlc['Close']
)])

fig.update_layout(
    title='AAPL Weekly Candlestick Chart',
    xaxis_title='Week',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False
)
fig.show()


In [ ]:
# Moving average signals (simple heuristic)
stock_data = stock_data.copy()
stock_data['MA_20'] = stock_data['Close'].rolling(window=20).mean()

buy_days = stock_data[stock_data['Close'] < stock_data['MA_20']]
sell_days = stock_data[stock_data['Close'] > stock_data['MA_20']]

plt.figure(figsize=(10, 5))
plt.plot(stock_data['Close'], label='Close')
plt.plot(stock_data['MA_20'], linestyle='--', label='20-day MA')
plt.scatter(buy_days.index, buy_days['Close'], label='Potential Buy', s=12)
plt.title('AAPL: Potential Buy Days (Close < 20-day MA)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(stock_data['Close'], label='Close')
plt.plot(stock_data['MA_20'], linestyle='--', label='20-day MA')
plt.scatter(sell_days.index, sell_days['Close'], label='Potential Sell', s=12)
plt.title('AAPL: Potential Sell Days (Close > 20-day MA)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# MultiIndex enhancement: Year-Month view + monthly returns
df = stock_data[['Close']].dropna().copy()
df['Year'] = df.index.year
df['Month'] = df.index.month
df = df.set_index(['Year', 'Month'], append=True).reorder_levels(['Year','Month','Date'])

# Monthly last close and % return
monthly_last = stock_data['Close'].resample('M').last()
monthly_return = monthly_last.pct_change().dropna()

monthly_summary = pd.DataFrame({
    'MonthEnd_Close': monthly_last,
    'Monthly_Return_%': (monthly_return*100)
}).dropna()

monthly_summary

# Best / worst months (by return)
top3 = monthly_summary.sort_values('Monthly_Return_%', ascending=False).head(3)
bottom3 = monthly_summary.sort_values('Monthly_Return_%', ascending=True).head(3)
display(top3, bottom3)
